In [36]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 300
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
import re

In [37]:
# Model Opening
model_path = 'SIE'
model_ver = 'SIE_POS+FLUX'

if 'POS+FLUX' in model_ver:
    constraint = 'pos_flux'
elif 'POS' in model_ver:
    constraint = 'pos'

In [38]:
# Load the data
with open(model_path + '/' + model_ver + '_optresult' + '.dat', 'r') as file:
    opt_result = file.readlines()

opt_result

['------------------------------------------\n',
 'optimize ndim=7\n',
 'run 1: 116 lens models calculated\n',
 'chi^2 = 3.593754e+03  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 3.593754e+03 1.896385e+02 3.404115e+03 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.579661e+02  2.080973e+01  2.075958e+01  1.275745e-01 -4.421324e+00  0.000000e+00  0.000000e+00 \n',
 'point  1.0000  2.081279e+01  2.078042e+01 \n',
 '------------------------------------------\n',
 '------------------------------------------\n',
 'optimize ndim=7\n',
 'run 2: 24 lens models calculated\n',
 'chi^2 = 3.593754e+03  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 3.593754e+03 1.896385e+02 3.404115e+03 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.0

In [39]:
# Find the last line with 'optimize' in it
last_optimize_index = None
for idx in range(len(opt_result) - 1, -1, -1):
    if 'optimize' in opt_result[idx]:
        last_optimize_index = idx
        last_optimize_line = opt_result[idx]
        break
if last_optimize_index is None:
    raise ValueError("No line with 'optimize' found in the file.")

print(f"Last 'optimize' line found at index: {last_optimize_index}")

# Extract everything after the last 'optimize' line
opt_result = opt_result[last_optimize_index + 1:]

opt_result


Last 'optimize' line found at index: 15


['run 2: 24 lens models calculated\n',
 'chi^2 = 3.593754e+03  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 3.593754e+03 1.896385e+02 3.404115e+03 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.579661e+02  2.080973e+01  2.075958e+01  1.275745e-01 -4.421324e+00  0.000000e+00  0.000000e+00 \n',
 'point  1.0000  2.081279e+01  2.078042e+01 \n',
 '------------------------------------------\n']

In [47]:
# Count the number of lines that start with 'lens'
lens_count = sum(1 for line in opt_result if line.startswith('lens'))
print(f"Number of 'lens' lines: {lens_count}")

# Initialize a dictionary to hold the lens parameters
lens_params_dict = {}

# Extract the lens parameters
lens_params = []
for line in opt_result:
    if line.startswith('lens'):
        # Extract the line and split it into parts
        parts = re.split(r'\s+', line.strip())
        print(parts)
        # Extract the lens name and parameters
        lens_name = parts[1]
        params = [float(x) for x in parts[2:]]

        # Store the parameters in the dictionary
        lens_params_dict[lens_name] = params
        lens_params.append((lens_name, params))

Number of 'lens' lines: 1
['lens', 'sie', '0.2613', '1.579661e+02', '2.080973e+01', '2.075958e+01', '1.275745e-01', '-4.421324e+00', '0.000000e+00', '0.000000e+00']


In [46]:
lens_params_dict

{'sie': [0.2613, 157.9661, 20.80973, 20.75958, 0.1275745, -4.421324, 0.0, 0.0]}

In [ ]:
with open(model_path + '/' + constraint + '_point' + '.py', 'r') as file:
    py = file.readlines()

py